In [1]:
import csv
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata

In [156]:
!ls /home/ljb/Downloads/*.csv

 /home/ljb/Downloads/job_27469ef0-e146-4aac-a655-575a1013fe02.csv
 /home/ljb/Downloads/job_3328ed71-e01b-47c4-a2a9-be6276a59f08.csv
 /home/ljb/Downloads/job_3c081b1c-2215-42a1-bd7f-6e7395f087ad.csv
 /home/ljb/Downloads/job_71913fe0-d7bd-4740-a771-8afcd1356415.csv
 /home/ljb/Downloads/job_98d07a69-caba-4315-b14d-490ad5c2759a.csv
 /home/ljb/Downloads/job_basic_turtle_e21d6798-bbac-442b-aaff-0068876d3037.csv
 /home/ljb/Downloads/job_c95bcca3-80fb-470c-ba98-31dfbdf7188f.csv
 /home/ljb/Downloads/job_cbc0696c-a99e-4993-9bed-c6696aaf79e0.csv
 /home/ljb/Downloads/job_ecded33e-9ac1-4901-80de-4d105d3041f4.csv
 /home/ljb/Downloads/job_eeaf7eac-110e-4594-97ad-c4659229169e.csv
 /home/ljb/Downloads/job_fd760c83-3db1-4595-b6e2-7ad2a02cca52.csv
 /home/ljb/Downloads/job_rover_trajectory_50b2e076-26f4-439a-9861-e8695e6afdb5.csv
 /home/ljb/Downloads/job_rover_trajectory_cbc0696c-a99e-4993-9bed-c6696aaf79e0.csv
 /home/ljb/Downloads/job_turtle_grid_28b72b2e-9fc6-4c1d-8d7a-b5f97cfe70ab.csv
 /home/ljb/Downlo

In [160]:
results = "/home/ljb/Downloads/job_turtle_repeat_571f7c3f-b252-44f6-ab13-55611aaf9ebb.csv"  # turtle_repeat 50
results = "/home/ljb/Downloads/job_turtle_grid_28b72b2e-9fc6-4c1d-8d7a-b5f97cfe70ab.csv"  # turtle_grid 100

In [161]:
df = pd.read_csv(results)
# create a proper index and sort results
df.sort_values(
    by='scenario_id', 
    key=lambda x: x.str.split("_", expand=True)[0].astype(int),
    ignore_index=True,
    inplace=True
)
df.head()

,scenario_id,scenario_name,param:test/odom_tuning_theta,param:test/odom_tuning_forward,param:test/start_pose,param:test/segment_length,metric:error_orientation_final_deg,metric:cumulative_distance_final_m,metric:error_horizontal_final_m,metric:time_delta_max_ms
0,0_turtle_gridsearch,turtle_gridsearch,0.0010,0.001,"[1, 1, 0]",5,0.114829,20.32,0.023769,0.318527
1,1_turtle_gridsearch,turtle_gridsearch,0.0025,0.001,"[1, 1, 0]",5,0.527024,20.32,0.081338,0.350714
2,2_turtle_gridsearch,turtle_gridsearch,0.0050,0.001,"[1, 1, 0]",5,1.374439,20.32,0.191239,0.367641
3,3_turtle_gridsearch,turtle_gridsearch,0.0100,0.001,"[1, 1, 0]",5,2.095167,20.32,0.266254,0.586510
4,4_turtle_gridsearch,turtle_gridsearch,0.0250,0.001,"[1, 1, 0]",5,3.506148,20.48,0.429474,0.652313


In [162]:
# assumptions: 
# each metric name starts with 'metric:'
# each metric name ends with underscore + the metric's unit: '_deg'
metrics = sorted([m for m in df.columns if 'metric' in m])    # metric:time_delta_max_ms
metric_names = [m.split("metric:")[-1] for m in metrics]      # time_delta_max_ms
metric_units = [f"{n.split('_')[-1]}" for n in metric_names]  # ms
metric_titles = [f"{' '.join(n.split('_')[:-1])} ({u})" for n, u in zip(metric_names, metric_units)]  # time delta max (ms)
metric_stds = [np.std(df[m]) for m in metrics]

fig = make_subplots(
    rows=1, 
    cols=len(metrics),
    subplot_titles=[f"{title} <br> std = {std:.1g} {unit}" for title, std, unit in zip(metric_titles, metric_stds, metric_units)]
)
for i, metric in enumerate(metrics):
    fig.add_trace(
        px.box(
            df, 
            y=metric, 
            points="all",
        )['data'][0],
        row=1, col=i+1
    )
fig.update_layout(template='presentation', title="Basic statistics for each metric")
fig.write_html("metric_statistics.html")
fig

In [150]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(df.index), y=df[metrics[1]], name=metric_titles[1], mode='markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=list(df.index), y=df[metrics[2]], name=metric_titles[2], mode='markers'),
    secondary_y=True,
)
fig.update_xaxes(title_text="scenario result index")
fig.update_yaxes(title_text=metric_units[1], secondary_y=False)
fig.update_yaxes(title_text=metric_units[2], secondary_y=True)

fig.update_layout(template='presentation', title_text="Metrics Comparison For Each Scenario of the Grid Search")
fig.write_html("results_metrics.html")
fig

In [165]:
def add_figure_of_merit(df, metrics, weights):
    "compute the weighted sum of metrics for row of the dataframe"
    df["figure_of_merit"] = df[metrics].apply(
        lambda x: sum([x[m] * w for m, w in zip(metrics, weights)]),
        axis=1
    )
    return df

In [166]:
metricss = metrics[1:3]
weights = [10, 1]
metricss

['metric:error_horizontal_final_m', 'metric:error_orientation_final_deg']

In [167]:
df = add_figure_of_merit(df, metricss, weights)
df.tail()

,scenario_id,scenario_name,param:test/odom_tuning_theta,param:test/odom_tuning_forward,param:test/start_pose,param:test/segment_length,metric:error_orientation_final_deg,metric:cumulative_distance_final_m,metric:error_horizontal_final_m,metric:time_delta_max_ms,figure_of_merit
95,95_turtle_gridsearch,turtle_gridsearch,0.050,0.5,"[1, 1, 0]",5,9.536433,20.24,1.011157,0.415564,19.648005
96,96_turtle_gridsearch,turtle_gridsearch,0.075,0.5,"[1, 1, 0]",5,21.086918,20.32,1.878585,0.380754,39.872767
97,97_turtle_gridsearch,turtle_gridsearch,0.100,0.5,"[1, 1, 0]",5,11.329037,20.32,1.544482,0.353813,26.773854
98,98_turtle_gridsearch,turtle_gridsearch,0.250,0.5,"[1, 1, 0]",5,53.103758,20.32,4.663723,0.353575,99.740984
99,99_turtle_gridsearch,turtle_gridsearch,0.500,0.5,"[1, 1, 0]",5,76.355930,20.32,6.492350,0.292301,141.279428


In [168]:
fig = px.scatter(df, x=df.index, y=df["figure_of_merit"], hover_data=["scenario_id"])
fig.update_layout(template='presentation', title_text="Figure of Merit For Each Scenario of the Grid Search")
fig.update_xaxes(title_text="scenario result index")
fig.update_yaxes(title_text="figure of merit")
fig.write_html("results_fom.html")
fig

In [169]:
fig = px.line(
    df.sort_values(by=['figure_of_merit'], ignore_index=True), 
    y="figure_of_merit", 
    hover_data=["scenario_id"],
    markers=True,
    template='presentation',
    title='Scenarios Sorted by Figure of Merit',
)
fig.update_xaxes(title_text="rank, lowest is best")
fig.update_yaxes(title_text="figure of merit")
fig.write_html("results_ranking.html")
fig

3D plot to show the evolution of the figure of merit as a function of the parameters

In [170]:
param_x = "param:test/odom_tuning_theta"
param_y = "param:test/odom_tuning_forward"
param_z = "figure_of_merit"

In [177]:
# create a uniform grid to support the surface plot
grid_x, grid_y = np.mgrid[0:max(df[param_x]):50j, 0:max(df[param_y]):50j]
# identify the data points
points = [(x, y) for x, y in zip(
    df[param_x].values,
    df[param_y].values
)]
# interpolate the grid from the data points
grid_z0 = griddata(
    points, 
    df[param_z], 
    (grid_x, grid_y), 
    method='nearest')

In [179]:
fig = go.Figure(data=[
    go.Surface(
        z=grid_z0, 
        x=grid_x, 
        y=grid_y
    )
])
fig.add_trace(
    go.Scatter3d(
        x=df[param_x], 
        y=df[param_y], 
        z=df[param_z],
        mode='markers'
    )
)
fig.update_layout(
    template='presentation', 
    title_text="Figure of Merit For Each Scenario of the Grid Search",
    scene_camera_eye=dict(x=-1.87, y=-0.88, z=0.64),

)
fig.update_scenes(
    xaxis_title_text=param_x.split('/')[-1],
    yaxis_title_text=param_y.split('/')[-1],
    zaxis_title_text=param_z,
    # xaxis_type='log',
    # yaxis_type='log',
    # zaxis_type='log',
)
fig.write_html("results_surface.html")